In [1]:
import os
import shutil
import pathlib
from glob import glob
import warnings

import earthpy as et
import earthpy.earthexplorer as etee
import earthpy.spatial as es
import geopandas as gpd
import geoviews as gv
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
import numpy as np
import pandas as pd
import rioxarray as rxr
import rioxarray.merge as rxrmerge


# Ignore FutureWarning coming from hvplot
warnings.simplefilter(action='ignore', category=FutureWarning)

data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME)
den_dir = os.path.join(data_dir, 'denver-neighborhoods')
ndvi_dir = os.path.join(data_dir, 'denver-green-space', 'processed')

for a_dir in [den_dir, ndvi_dir]:
        if not os.path.exists(a_dir):
                os.makedirs(a_dir)

In [15]:
den_path = os.path.join(den_dir, 'denver-neighborhoods.shp')

if not os.path.exists(den_path):
    den_url = (
        'https://www.denvergov.org/media/gis/DataCatalog/'
        'statistical_neighborhoods/shape/statistical_neighborhoods.zip')
    gpd.read_file(den_url).to_file(den_path)

den_gdf = gpd.read_file(den_path)

den_gdf

neigh_gdf = (
    den_gdf
    .set_index('NBHD_NAME')
     .loc[['Sloan Lake', 'West Colfax']]
)
neigh_gdf.total_bounds
# neigh_gdf = neigh_gdf.to_crs(3857)
# den_gdf = den_gdf.to_crs(3857)

array([-105.05324032,   39.7339687 , -105.02515184,   39.75851948])

In [13]:
gv.tile_sources.OSM * neigh_gdf.to_crs(3857).hvplot(
    line_color='Black', fill_color=None, line_width=3,
    xaxis=None, yaxis=None)

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [x,y]

In [17]:
bbox = etee.BBox(*neigh_gdf.total_bounds)
naip_downloader = etee.EarthExplorerDownloader(
    dataset="NAIP", 
    label='denver-green-space', 
    bbox=bbox,
    start='2019-01-01', 
    end='2019-12-31',
    store_credential=True)
naip_downloader.submit_download_request()
naip_downloader.download(override=False)

Login Successful.
Searching datasets...
Using dataset alias: naip
Searching scenes...
Found 2 scenes
4 products found.
Downloads staging...
C:\Users\tayob\earth-analytics\data\denver-green-space\M_3910516_SE_13_060_20190803.zip
Saving download: M_3910516_SE_13_060_20190803


BadZipFile: File is not a zip file

In [8]:
def download_neighborhood_data(name, geometry, start, end):
    """
    Download NAIP raster for a given geometry, start date, and end date

    Parameters
    ==========
    name : str
      The name used to label the download
    geometry : shapely.POLYGON
      The geometry to derive the download extent from. 
      Must have a `.bounds` attribute.
    start : str
      The start date as 'YYYY-MM-DD'
    end : str
      The end date as 'YYYY-MM-DD'

    Returns
    =======
    downloader : earthpy.earthexplorer.EarthExplorerDownloader
      Object with information about the download, including the data directory.
    """
    print(f'\nNeighborhood name: {name}')
    bbox = etee.BBox(*geometry.bounds)
    naip_downloader = etee.EarthExplorerDownloader(
        dataset="NAIP", 
        label=name.lower().replace(' ', '-'), 
        bbox=bbox,
        start=start, 
        end=end,
        store_credential=True)
    naip_downloader.submit_download_request()
    naip_downloader.download(override=False)
    return naip_downloader

ndvi_stats_path = os.path.join(ndvi_dir, 'neighborhood-ndvi-stats.csv')
if os.path.exists(ndvi_stats_path):
    print('Reading in NDVI statistics file...')
    ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col='neighborhood')
else:
    print('NDVI statistics file does not exist...')
    ndvi_stats_df = pd.DataFrame()

for neighborhood_name, details in neigh_gdf.iterrows():
    if neighborhood_name in ndvi_stats_df.index:
      print('Neighborhood stats have already been calculated. Skipping')
      continue
    downloader = download_neighborhood_data(
        neighborhood_name, details.geometry, '2019-01-01', '2019-12-31')

NDVI statistics file does not exist...

Neighborhood name: Sloan Lake
Login Successful.
Searching datasets...
Using dataset alias: naip
Searching scenes...
Found 0 scenes


TypeError: 'NoneType' object is not iterable

In [ ]:
def load_and_merge_arrays(name):
    """
    Load in and merge downloaded arrays

    Parameters
    ==========
    name : str
      The name used to label the download

    Returns
    =======
    merge_da : rxr.DataArray
        DataArray with the merged data
    """

    print(f'\nNeighborhood name: {name}')
    data_path = os.path.join(
        et.io.HOME, et.io.DATA_NAME,
        name.lower().replace(' ', '-'))
    tif_paths = glob(os.path.join(data_path, '*.tif'))
    das = [rxr.open_rasterio(tp, masked=True) for tp in tif_paths]
    merged_da = rxrmerge.merge_arrays(das)
    return merged_da

merged_da = load_and_merge_arrays('West Colfax')
merged_da

In [ ]:
def calculate_ndvi_statistics(gdf, da, stats_path, override=False):
    """
    Calculate NDVI, then summarize and save statistics

    Uses downloaded National Agricultural Imagery Program (NAIP)
    data. <citation>

    Parameters
    ==========
    gdf : gpd.GeoDataFrame
      One row with the neighborhood name and boundary
    da : rxr.DataArray
        Multispectral (NAIP) raster data
    stats_path: pathlike
        The path to the statistics file to save results
    """
    name = str(gdf.index[0])
    print(f'Neighborhood name: {name}')

    file_is_empty = True
    if os.path.exists(stats_path):
        print('Stats file exists.')
        stats_df = pd.read_csv(stats_path)
        file_is_empty = len(stats_df)==0
        print(f'Stats file is empty? {file_is_empty}')

        if not file_is_empty:
            if (name in list(stats_df.neighborhood)) and (not override):
                print('Stats already calculated. Skipping...')
                return
    
    reprojected_gdf = gdf.to_crs(da.rio.crs)

    naip_crop_da = da.rio.clip_box(*reprojected_gdf.total_bounds)
    naip_da = naip_crop_da.rio.clip(reprojected_gdf.geometry)

    ndvi_da = (
        (da.sel(band=4) - da.sel(band=1))
        / (da.sel(band=4) + da.sel(band=1))
    )
    
    mode = 'w' if file_is_empty else 'a'
    pd.DataFrame(dict(
        neighborhood=[name],
        ndvi_min=[float(ndvi_da.min())],
        ndvi_max=[float(ndvi_da.max())],
        ndvi_median=[float(ndvi_da.median())],
        ndvi_25pctl=[np.nanpercentile(ndvi_da.values, 25)],
        ndvi_75pctl=[np.nanpercentile(ndvi_da.values, 75)],
        ndvi_mean=[float(ndvi_da.mean())],
        ndvi_std=[float(ndvi_da.std())]
    )).to_csv(stats_path, mode='a', header=file_is_empty)

calculate_ndvi_statistics(
    den_gdf.loc[['West Colfax']], merged_da, ndvi_stats_path)

In [ ]:
ndvi_stats_path = os.path.join(ndvi_dir, 'neighborhood-ndvi-stats.csv')

for neighborhood_name, details in den_gdf.iterrows():
    if not os.path.exists(ndvi_stats_path):
        print('NDVI statistics file does not exist...')
        ndvi_stats_df = pd.DataFrame()
    else:
        ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col="neighborhood")
    
    if neighborhood_name in ndvi_stats_df.index:
      print('Neighborhood stats have already been calculated. Skipping')
      continue
    downloader = download_neighborhood_data(
        neighborhood_name, details.geometry, '2019-01-01', '2019-12-31')
    merged_da = load_and_merge_arrays(neighborhood_name)
    calculate_ndvi_statistics(
        den_gdf.loc[[neighborhood_name]], merged_da, ndvi_stats_path)
    
    
    shutil.rmtree(downloader.data_dir)

In [ ]:
ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col="neighborhood")
polygons = gv.Polygons(den_gdf.join(ndvi_stats_df, how='left'), vdims=['ndvi_mean'])
color_scale = 'viridis'
chloropleth = gv.tile_sources.StamenToner * polygons.opts(
    title='NDVI Mean Chloropleth',
    cmap=color_scale,
    colorbar=True
)
hv.extension('bokeh')
hv.save(chloropleth, 'chloropleth_plot.html')